In [1]:
import yfinance as yf
import pandas as pd

ticker = "F"
ul_price = yf.Ticker(ticker).history(period='1d')['Close'][0]
expirations = yf.Ticker(ticker).options[:3]

calls = pd.concat( [yf.Ticker(ticker).option_chain(expirations[0]).calls, yf.Ticker(ticker).option_chain(expirations[1]).calls, yf.Ticker(ticker).option_chain(expirations[2]).calls])

calls['intrinsic_value'] = ul_price - calls['strike'] 
calls.loc[calls['intrinsic_value'] < 0, 'intrinsic_value'] = 0
calls['market_price'] = (calls['bid'] + calls['ask']) / 2
calls['extrinsic_value'] = calls['market_price'] - calls['intrinsic_value']
calls['extrinsic_to_strike_percentage'] = calls['extrinsic_value'] / calls['strike'] 
calls['extrinsic_to_underlying_percentage'] = calls['extrinsic_value'] / ul_price
calls['called_away_percentage'] = ( ul_price - calls['market_price'] - calls['strike']) / ul_price * -1

print(ul_price)
print(calls[calls['extrinsic_value'] > .1].to_markdown())

13.260000228881836
|    | contractSymbol   | lastTradeDate             |   strike |   lastPrice |   bid |   ask |   change |   percentChange |   volume |   openInterest |   impliedVolatility | inTheMoney   | contractSize   | currency   |   intrinsic_value |   market_price |   extrinsic_value |   extrinsic_to_strike_percentage |   extrinsic_to_underlying_percentage |   called_away_percentage |
|---:|:-----------------|:--------------------------|---------:|------------:|------:|------:|---------:|----------------:|---------:|---------------:|--------------------:|:-------------|:---------------|:-----------|------------------:|---------------:|------------------:|---------------------------------:|-------------------------------------:|-------------------------:|
|  7 | F230804C00013000 | 2023-07-28 19:59:56+00:00 |     13   |        0.39 |  0.36 |  0.39 |    -0.57 |        -59.375  |    12123 |           1421 |            0.306648 | True         | REGULAR        | USD        |         

In [2]:
xxx = yf.Ticker(ticker).history(period='1d')['Close'][0]
print(xxx)

13.260000228881836


In [ ]:
import json
print(json.dumps(yf.Ticker(ticker).info, indent=4) )

In [26]:
import yfinance as yf
import pandas as pd
import datetime # import the datetime module

ticker = yf.Ticker("HUT") 
commish = 1
today = pd.to_datetime(datetime.date.today()) # get today's date as a DatetimeArray object


exp_dates = ticker.options # get all expiration dates
max_puts = [] # create an empty list to store the max_put for each date
today_data = ticker.history(period='1d') # get today's data
close_price = today_data['Close'].iloc[0] # get the close price
for date in exp_dates:
  opt = ticker.option_chain(date) # get option chain for a given date
  
  puts = opt.puts[(opt.puts['bid'] > 0) & (opt.puts['openInterest'] > 0)].copy() # get put options data and exclude puts that have a bid of 0
  
  puts.loc[:, 'lastTradeDate'] = pd.to_datetime(puts['lastTradeDate']) # convert the lastTradeDate column to a datetime object using .loc method
  puts.loc[:, 'lastTradeDate'] = puts['lastTradeDate'].dt.tz_localize(None) # remove the timezone information using .dt.tz_localize(None) method
  puts = puts[puts['lastTradeDate'] >= today - pd.Timedelta(days=30)] # filter out the puts that have not been traded in the last 30 days using .loc method


  # calculate extrinsic value and find the highest one
  puts.loc[:, 'mid'] = (puts['bid'] + puts['ask']) / 2 # calculate the mid point between the bid and ask using .loc method
  puts.loc[:, 'intrinsic'] = puts['strike'] - close_price # calculate intrinsic value using the close price using .loc method
  puts.loc[:, 'intrinsic'] = puts['intrinsic'].clip(lower=0) # ensure that intrinsic value is not less than 0 using .clip method
  puts.loc[:, 'extrinsic'] = puts['mid'] - puts['intrinsic'] # calculate extrinsic value using the mid point using .loc method
  max_extrinsic = puts['extrinsic'].max() # find the maximum extrinsic value
  max_put = puts[puts['extrinsic'] == max_extrinsic] # find the put option with the maximum extrinsic value
  max_puts.append(max_put) # add the max_put to the list


max_puts_df = pd.concat(max_puts) # concatenate the list of dataframes into a single dataframe

ticker_len = len(ticker.ticker) # get the length of the ticker symbol

max_puts_df.loc[:, 'expiryDate'] = pd.to_datetime(max_puts_df['contractSymbol'].str.slice(ticker_len, ticker_len+6), format='%y%m%d').dt.date # create a column named expiryDate using the contractSymbol column and convert it to a date object
max_puts_df.loc[:, 'dte'] = (pd.to_datetime(max_puts_df['expiryDate']) - today).dt.days # create a column named dte using the expiryDate column and calculate the days to expiration as an int

max_puts_df = max_puts_df[~(max_puts_df['dte'] == 0)] # remove rows where dte is 0

max_puts_df.loc[:, 'simpleReturn'] = (max_puts_df['extrinsic'] - commish/100) / max_puts_df['strike'] # calculate the simple return using the extrinsicValue and strike columns
max_puts_df.loc[:, 'annualized'] = max_puts_df['simpleReturn'] / max_puts_df['dte'] * 365  # calculate the annualized return using the extrinsicValue and strike columns

xxx_df = max_puts_df.loc[:, ['dte', 'expiryDate', 'strike', 'bid', 'ask', 'mid', 'extrinsic', 'simpleReturn', 'annualized', 'impliedVolatility']]
print(xxx_df.to_markdown())

#print(max_puts_df.to_markdown()) # print the updated dataframe as a markdown table


|    |   dte | expiryDate   |   strike |   bid |   ask |   mid |   extrinsic |   simpleReturn |   annualized |   impliedVolatility |
|---:|------:|:-------------|---------:|------:|------:|------:|------------:|---------------:|-------------:|--------------------:|
|  3 |     7 | 2023-08-04   |      3.5 |  0.2  |  0.21 | 0.205 |       0.185 |      0.05      |     2.60714  |            0.945313 |
|  3 |    14 | 2023-08-11   |      3.5 |  0.3  |  0.35 | 0.325 |       0.305 |      0.0842857 |     2.19745  |            1.11719  |
|  5 |    21 | 2023-08-18   |      3.5 |  0.38 |  0.41 | 0.395 |       0.375 |      0.104286  |     1.81259  |            1.12891  |
|  4 |    28 | 2023-08-25   |      3.5 |  0.44 |  0.49 | 0.465 |       0.445 |      0.124286  |     1.62015  |            1.16407  |
|  3 |    35 | 2023-09-01   |      3.5 |  0.48 |  0.56 | 0.52  |       0.5   |      0.14      |     1.46     |            1.17188  |
|  3 |    49 | 2023-09-15   |      3.5 |  0.56 |  0.59 | 0.575 |     